In [30]:
import numpy as np

from data.test_dataset import test_training_labels
from data.dataset import Dataset
from preprocessing.constants import TRAINING_PATH, TESTING_PATH

ImportError: cannot import name 'test_training_labels' from 'data.test_dataset' (/Users/brookeryan/Developer/BaldiLab/Visual-Acuity/data/test_dataset.py)

In [2]:
print("HI33316")

HI33316


In [3]:
training_dataset = Dataset(TRAINING_PATH, name="Training", height=400, width=400, batch_size=30)
testing_dataset = Dataset(TESTING_PATH, name="Training", height=400, width=400, batch_size=30)

Training Dataset contains 8478 images
SIZE =  8478
Training Dataset contains 8255 images
SIZE =  8255


In [29]:
training_dataset.process_labels()

In [28]:
test_training_labels(training_dataset)

AttributeError: module 'data.test_dataset' has no attribute 'test_training_labels'

In [ ]:
print("~~~~~~~~~~~~")
print("    Test    ")
print("~~~~~~~~~~~~")
print("y_test_optotype: ", testing_dataset.optotypes)
print("y_test_angle: ", np.unique(training_dataset.angles))

In [ ]:
import matplotlib.pyplot as plt
def print_random_test_values(ax):
    index = np.random.randint(0, x_test.shape[0])
    print("~~~~~~~~~~~~")
    print("Test Values ")
    print("~~~~~~~~~~~~")

    print("Index = {x}".format(x=index))

    print("Optotype = {x}".format(x=y_test_optotype[index]))
    print("Angle = {x}".format(x=y_test_angle[index]))
    print(x_test[index])

    ax.set_title("Test- {x}".format(x=y_test_optotype[index]))
    ax.imshow(x_test[index])

def print_random_training_values(ax):
    index = np.random.randint(0, x_train.shape[0])
    print("~~~~~~~~~~~~")
    print("Train Values")
    print("~~~~~~~~~~~~")

    print("Index = {x}".format(x=index))

    print("Optotype = {x}".format(x=y_train_optotype[index]))
    print("Angle = {x}".format(x=y_train_angle[index]))
    print(x_test[index])

    ax.set_title("Train - {x}".format(x=y_train_optotype[index]))
    ax.imshow(x_train[index])

fig, ax = plt.subplots(1, 2, figsize=(10, 10))
print_random_test_values(ax[0])
print_random_training_values(ax[1])

plt.show()

## Transfer learning

## RESNET50

In [ ]:
from tensorflow import keras


TODO : Going to try this experiment with this guy https://github.com/EscVM/Efficient-CapsNet.

In [ ]:
def testicle_test():
    y_train_angle = np.zeros(27, dtype=np.int)
    for i, sample_name in enumerate(low_distortion_filenames):
        thing = extract_angle(sample_name)
        y_train_angle[i] = thing
    print(np.unique(y_train_angle))

testicle_test()

# Transfer learning with VGG16

### Create the model
- Make sure `model.trainable = False `
-

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

training_set = image_dataset_from_directory("/Users/brookeryan/Developer/BaldiLab/Visual-Acuity/images/training/",
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(150, 150))
val_dataset = image_dataset_from_directory("/Users/brookeryan/Developer/BaldiLab/Visual-Acuity/images/testing/",
                                                  shuffle=True,
                                                  batch_size=32,
                                                  image_size=(150, 150))

In [ ]:
def create_base_model():
    vgg = VGG16(include_top=False, weights="imagenet", input_shape=(h, w, 3))
    vgg.trainable = False
    return vgg

base_model = create_base_model()
x = base_model(x_train, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
inputs = keras.Input(shape=(h, w, 3))
outputs = keras.layers.Dense(1, activation=tf.keras.activations.softmax)(x)
model = keras.Model(inputs, outputs)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(training_set, epochs=20, validation_data=val_dataset)

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
import ssl

ssl._create_default_https_context = ssl._create_unverified_context
# !pip install pydot
# !pip install graphviz
# !pip install pydotplus

model = tf.keras.applications.VGG16()
# plot_model(model)

model.summary()

In [ ]:
from keras import  models

base_model = VGG16(weights='imagenet')
model_VGG16 = models.Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)

model_VGG16.summary()

In [ ]:
model = VGG16(include_top=False, weights="imagenet", input_shape=(h, w, 3))
model.summary()

In [ ]:
for layer in model.layers[:45]:
    layer.trainable = False
    print(layer)

In [ ]:
import tensorflow.keras as K

def create_transfer_model_2():
    input_t = K.Input(shape=(h, w, 3))
    res_model = K.applications.ResNet50(include_top=False,
                                        weights="imagenet",
                                        input_tensor=input_t)

    for layer in res_model.layers[:45]:
        layer.trainable = False
    #to_res = (224, 224)
    model = K.models.Sequential()
    #model.add(K.layers.Lambda(lambda image: tf.image.resize(image, to_res)))
    model.add(res_model)
    model.add(K.layers.Flatten())
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(256, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(128, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(64, activation='relu'))
    model.add(K.layers.Dropout(0.5))
    model.add(K.layers.BatchNormalization())
    model.add(K.layers.Dense(4, activation='softmax')) #this

    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])

    return model

model = create_transfer_model_2()
model.summary()